## ChatGLM-6B 介绍  

ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 [General Language Model (GLM)](https://github.com/THUDM/GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。  
ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答，更多信息请参考我们的[博客](https://chatglm.cn/blog)。  

为了方便下游开发者针对自己的应用场景定制模型，我们同时实现了基于 [P-Tuning v2](https://github.com/THUDM/P-tuning-v2) 的高效参数微调方法 [(使用指南)](ptuning/README.md) ，INT4 量化级别下最低只需 7GB 显存即可启动微调。  

不过，由于 ChatGLM-6B 的规模较小，目前已知其具有相当多的[**局限性**](#局限性)，如事实性/数学逻辑错误，可能生成有害/有偏见内容，较弱的上下文能力，自我认知混乱，以及对英文指示生成与中文指示完全矛盾的内容。请大家在使用前了解这些问题，以免产生误解。更大的基于 1300 亿参数 [GLM-130B](https://github.com/THUDM/GLM-130B) 的 ChatGLM 正在内测开发中。  

[Github链接](https://github.com/THUDM/ChatGLM-6B)

## 项目介绍  

本项目提供了 ChatGLM-6B 非量化版本在 ModelWhale 平台 V100 机型上进行部署、推理测试以及微调的教程  

### 环境配置  
计算资源：V100 Tensor Core GPU  
镜像：Cuda11.3.1 PyTorch 1.10.2 Tensorflow2.8.0 Python 3.7 镜像  
如果希望持久化保存 Fine-tuning 结果，请选择 **挂载 work 目录**  

**官方注**：ModelWhale GPU机型需要从云厂商拉取算力资源，耗时5~10min，且会预扣半小时资源价格的鲸币。如果资源未启动成功，预扣费用会在关闭编程页面后五分钟内退回，无需紧张，如遇问题欢迎[提报工单](https://www.heywhale.com/home/user/workorder)，客服会及时处理。

项目介绍
本项目提供了 ChatGLM-6B 非量化版本在 ModelWhale 平台 V100 机型上进行部署、推理测试以及微调的教程



In [1]:
# 确认GPU型号与显存大小
!nvidia-smi

Tue Nov 21 16:24:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:10.0 Off |                  N/A |
| 63%   83C    P2   342W / 350W |   6202MiB / 24576MiB |     74%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# 检查模型文件正确挂载
# !ls -lh /root/sunyd/codes/ChatGLM-6B 

总用量 152K
-rw-r--r-- 1 root root 1.9K Nov 21 14:39 api.py
-rw-r--r-- 1 root root 1.9K Nov 21 14:39 cli_demo.py
-rw-r--r-- 1 root root 2.0K Nov 21 14:39 cli_demo_vision.py
drwxr-xr-x 2 root root 4.0K Nov 21 14:39 examples
-rw-r--r-- 1 root root  940 Nov 21 14:39 FAQ.md
drwxr-xr-x 2 root root 4.0K Nov 21 14:39 improve
-rw-r--r-- 1 root root  12K Nov 21 14:39 LICENSE
drwxr-xr-x 2 root root 4.0K Nov 21 14:39 limitations
-rw-r--r-- 1 root root 4.2K Nov 21 14:39 MODEL_LICENSE
-rw-r--r-- 1 root root 4.6K Nov 21 14:39 PROJECT.md
drwxr-xr-x 5 root root 4.0K Nov 21 15:54 ptuning
-rw-r--r-- 1 root root  22K Nov 21 14:39 README_en.md
-rw-r--r-- 1 root root  25K Nov 21 14:39 README.md
-rw-r--r-- 1 root root   96 Nov 21 14:39 requirements.txt
drwxr-xr-x 2 root root 4.0K Nov 21 14:39 resources
-rw-r--r-- 1 root root 6.3K Nov 21 14:39 UPDATE.md
-rw-r--r-- 1 root root 2.0K Nov 21 14:39 utils.py
-rw-r--r-- 1 root root 2.2K Nov 21 14:39 web_demo2.py
-rw-r--r-- 1 root root 2.0K Nov 21 14:39 web_demo_old.py

In [2]:
# !ls -lh /root/sunyd/model_hub/chatglm-6b/ 

总用量 13G
-rw-r--r-- 1 root root   773 Jun 26 20:22 config.json
-rw-r--r-- 1 root root  4.2K Jun 26 20:22 configuration_chatglm.py
-rw-r--r-- 1 root root  2.6M Jun 26 20:22 ice_text.model
-rw-r--r-- 1 root root   57K Jun 26 20:22 modeling_chatglm.py
-rw-r--r-- 1 root root  1.7G Jul 14 23:48 pytorch_model-00001-of-00008.bin
-rw-r--r-- 1 root root  1.8G Jul 14 23:51 pytorch_model-00002-of-00008.bin
-rw-r--r-- 1 root root  1.9G Jul 14 23:55 pytorch_model-00003-of-00008.bin
-rw-r--r-- 1 root root  1.8G Jul 15 00:01 pytorch_model-00004-of-00008.bin
-rw-r--r-- 1 root root  1.8G Jul 15 00:07 pytorch_model-00005-of-00008.bin
-rw-r--r-- 1 root root  1.8G Jul 15 00:12 pytorch_model-00006-of-00008.bin
-rw-r--r-- 1 root root  1.1G Jul 15 00:23 pytorch_model-00007-of-00008.bin
-rw-r--r-- 1 root root 1020M Jul 15 00:34 pytorch_model-00008-of-00008.bin
-rw-r--r-- 1 root root   33K Jun 26 20:22 pytorch_model.bin.index.json
-rw-r--r-- 1 root root   15K Jun 26 20:22 quantization.py
-rw-r--r-- 1 root root 

## 环境准备完毕
> 代码位置：/home/mw/project/ChatGLM-6B  
> 模型位置：/home/mw/input/ChatGLM6B6449